##  準備

In [1]:
from pathlib import Path
from bs4 import BeautifulSoup
import json
import os
import re
import time
import logging
import pandas as pd
!pip install selenium
!pip install beautiful requests
!pip install BeautifulSoup4
from selenium import webdriver
from selenium.webdriver.support.select import Select
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from time import sleep
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert


from selenium import webdriver
from selenium.webdriver.support.select import Select
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from time import sleep
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.webdriver.common.alert import Alert

pd.set_option('display.max_rows', 100)

  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
requests 2.22.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.16 which is incompatible.
cookiecutter 1.7.2 requires requests>=2.23.0, but you have requests 2.22.0 which is incompatible.
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.16
    Uninstalling urllib3-1.26.16:
      Successfully uninstalled urllib3-1.26.16
ERROR: pip's dependency resolv

In [2]:
pip install chromedriver-binary-auto

Note: you may need to restart the kernel to use updated packages.


In [11]:

options = webdriver.ChromeOptions()
options.binary_location = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
chrome_driver_binary = "./chromedriver"
browser1 = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
#browser1.execute_script("alert('running javascript');")
# 終了したい場合
# browser1.quit()

/var/folders/_q/5b0cfy_d2ks1qlbxzgflbk880000gn/T/ipykernel_27494/1770538417.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser1 = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
/var/folders/_q/5b0cfy_d2ks1qlbxzgflbk880000gn/T/ipykernel_27494/1770538417.py:4: DeprecationWarning: use options instead of chrome_options
  browser1 = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


In [12]:

url = "https://tango.naragakuen-u.jp/aa_web/syllabus/se0010.aspx?me=EU&opi=mt0010"

browser1.get(url)


In [56]:
colleage_name = "大阪大谷大学(学部)"
# ディレクトリが存在しない場合に作成
if not os.path.exists(colleage_name):
    os.mkdir(colleage_name)


In [31]:

table = browser1.find_element(By.ID, "ctl00_cphMain_grdSyllabusList")
serach_list = table.find_elements(By.CLASS_NAME, "serach_list")
serach_list_alter = table.find_elements(By.CLASS_NAME, "serach_list_alter")
serach_list[1].find_element(By.TAG_NAME,"a").click()

In [58]:
browser1.switch_to.window(browser1.window_handles[0])
#検索を行う
search_button = browser1.find_element(By.ID,"ctl00_cphMain_ibtnSearch")
search_button.click()
sleep(3)



count_classes = 0
loop = True
while loop:

    pre_table = browser1.find_element(By.ID, "ctl00_cphMain_grdSyllabusList")
    pre_serach_list = pre_table.find_elements(By.CLASS_NAME, "serach_list")
    pre_serach_list_alter = pre_table.find_elements(By.CLASS_NAME, "serach_list_alter")

    for index in range(0,len(pre_serach_list)):
        table = browser1.find_element(By.ID, "ctl00_cphMain_grdSyllabusList")
        serach_list = table.find_elements(By.CLASS_NAME, "serach_list")

        class_name = result = str(count_classes).zfill(8) + serach_list[index].text
        count_classes = count_classes + 1
        

        a_link =serach_list[index].find_elements(By.TAG_NAME,"a")
        if len(a_link)!=1:
            continue
        sleep(0.1)
        if os.path.isfile("./"+colleage_name+"/"+ class_name.replace("/", "_")+ ".html"):
            print("skip " + class_name)
        else:
            a_link[0].click()
            sleep(1.5)
            html = browser1.page_source
            with open("./"+colleage_name+"/"+ class_name.replace("/", "_")+ ".html", "w") as file:
                file.write(html)
            browser1.back()
            sleep(1)

    for index in range(0,len(pre_serach_list_alter)):
        table = browser1.find_element(By.ID, "ctl00_cphMain_grdSyllabusList")
        serach_list_alter = table.find_elements(By.CLASS_NAME, "serach_list_alter")

        class_name = result = str(count_classes).zfill(8) + serach_list_alter[index].text
        count_classes = count_classes + 1

        a_link =serach_list_alter[index].find_elements(By.TAG_NAME,"a")

 
        sleep(0.1)
        if os.path.isfile("./"+colleage_name+"/"+ class_name.replace("/", "_")+ ".html"):
            print("skip " + class_name)
        else:
            a_link[0].click()
            sleep(1.5)
            html = browser1.page_source
            with open("./"+colleage_name+"/"+ class_name.replace("/", "_")+ ".html", "w") as file:
                file.write(html)
            print(class_name)
            browser1.back()
            sleep(1)

    next_button = browser1.find_elements(By.NAME,"ctl00$cphMain$grdSyllabusList$ctl01$ctl10")
    next_button.extend(browser1.find_elements(By.NAME,"ctl00$cphMain$grdSyllabusList$ctl54$ctl09"))
    next_button.extend(browser1.find_elements(By.NAME,"ctl00$cphMain$grdSyllabusList$ctl01$ctl09"))
    next_button.extend(browser1.find_elements(By.NAME,"ctl00$cphMain$grdSyllabusList$ctl24$ctl09"))
    next_button.extend(browser1.find_elements(By.NAME,"ctl00$cphMain$grdSyllabusList$ctl24$ctl11"))
    next_button.extend(browser1.find_elements(By.NAME,"ctl00$cphMain$grdSyllabusList$ctl01$ctl11"))
    if len(next_button) > 0:
        next_button[0].click()
    else:
        print("end of syllabus")
        loop = False
        #browser1.quit()

skip 0000000111001 心理学A 前期 2単位 前期: 月曜1限 文1
田沢　晶子
skip 0000000211003 生涯発達と教育 前期 2単位 前期: 月曜1限 教育1(64)、教育1(65)、教育1(66)、人社2
四辻　伸吾
skip 0000000311005 臨床心理学概論 前期 2単位 前期: 月曜1限 人間1
田沢　晶子
skip 0000000411007 数学・統計学 前期 2単位 前期: 月曜1限 薬学2
宇田川　周子、森本　正太郎
skip 0000000511009 幼児教育課程論 前期 2単位 前期: 月曜1限 教育2(6S)
井上　美智子
skip 0000000611011 音楽科教育法 前期 2単位 前期: 月曜1限 教育2(64)
長谷川　諒
skip 0000000711014 介護概論 前期 2単位 前期: 月曜1限 人社2
植木　是
skip 0000000811016 美術史学実習Ⅰ 前期 1単位 前期: 月曜1限 歴文2
高志　緑
skip 0000000911018 保健体育科教育法B 後期 2単位 後期: 月曜1限 スポ2
宮内　一三
skip 0000001011020 薬理学A 後期 2単位 後期: 月曜1限 薬学2
田邉　彬恵、水口　博之、綿野　智一
skip 0000001111002 教育原理 前期 2単位 前期: 月曜1限 教育1(61)、教育1(62)、教育1(63)
江上　直樹
skip 0000001211004 博物館資料保存論 後期 2単位 後期: 月曜1限 文1、教育1、人間1
高志　緑
skip 0000001311006 スポーツアナリスト論 前期 2単位 前期: 月曜1限 スポ1
泉　健介
skip 0000001411008 日本文学史Ⅰ 前期 2単位 前期: 月曜1限 教育2(64)、教育2(65)、教育2(66)
荒井　洋樹
skip 0000001511010 保育内容（人間関係） 前期 2単位 前期: 月曜1限 教育2(66)、教育2(6K)
井上　寿美
skip 0000001611013 特別活動の指導法 後期 2単位 後期: 月曜1限 教育2
藤原　靖浩
skip 0000001711015 子どもの健康と安全 後期 2単位 後期: 月曜1限 教育2(6

In [52]:
        table = browser1.find_element(By.ID, "ctl00_cphMain_grdSyllabusList")
        serach_list = table.find_elements(By.CLASS_NAME, "serach_list")
        serach_list

[<selenium.webdriver.remote.webelement.WebElement (session="cb988e770c6c480adee93175e8c3c68c", element="EDB8D251288891D0D603D5DAE3046A27_element_36693")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cb988e770c6c480adee93175e8c3c68c", element="EDB8D251288891D0D603D5DAE3046A27_element_36694")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cb988e770c6c480adee93175e8c3c68c", element="EDB8D251288891D0D603D5DAE3046A27_element_36695")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cb988e770c6c480adee93175e8c3c68c", element="EDB8D251288891D0D603D5DAE3046A27_element_36696")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cb988e770c6c480adee93175e8c3c68c", element="EDB8D251288891D0D603D5DAE3046A27_element_36697")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cb988e770c6c480adee93175e8c3c68c", element="EDB8D251288891D0D603D5DAE3046A27_element_36698")>,
 <selenium.webdriver.remote.webelement.WebElement (session="cb988e770c